# Lab #1 (Part 4): 서울시 따릉이 수요 예측 GBMs 모델 성능 비교

지금까지 배운 Gradient Boosting 모델 3대장(XGBoost, LightGBM, CatBoost)을 모두 사용하여 '서울 따릉이' 수요 예측 문제를 해결해보고, 최종 성능을 비교 분석하여 최적의 모델을 찾아봅시다.

### 과제 목표

1.  간단한 특성 공학(Feature Engineering)을 수행합니다.
2.  XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고 성능(RMSE, R²)을 평가합니다.
3.  세 모델의 성능을 시각적으로 비교하고 결과를 해석합니다.

### Step 1: 데이터 준비 및 특성 공학

기존에 사용한 `df_orig` 데이터프레임에 새로운 시간 관련 특성을 추가해 봅시다.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as cb


df_final = pd.read_csv('./datasets/SeoulBikeData.csv', encoding='cp949')

df_final.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                    'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                    'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

In [ ]:
# 원본 데이터 로드
df_final = pd.read_csv('../datasets/ml/bike-sharing/SeoulBikeData.csv', encoding='cp949')

# 컬럼명 정리
df_final.columns = ['Date', 'Rented_Bike_Count', 'Hour', 'Temperature', 'Humidity',
                    'Wind_Speed', 'Visibility', 'Dew_Point_Temp', 'Solar_Radiation',
                    'Rainfall', 'Snowfall', 'Seasons', 'Holiday', 'Functioning_Day']

# 📌 특성 공학: 주중/주말, 출퇴근 시간 특성 추가
df_final['Date'] = pd.to_datetime(df_final['Date'], format='%d/%m/%Y')
df_final['Weekday'] = df_final['Date'].dt.weekday # 0:월, 6:일
df_final['Is_Weekend'] = df_final['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
df_final['Is_Rush_Hour'] = df_final['Hour'].apply(lambda x: 1 if (x >= 7 and x <= 9) or (x >= 17 and x <= 19) else 0)

# 사용할 특성 정의
features = ['Hour', 'Temperature', 'Humidity', 'Wind_Speed', 'Visibility',
            'Solar_Radiation', 'Rainfall', 'Snowfall', 'Seasons', 'Holiday',
            'Functioning_Day', 'Is_Weekend', 'Is_Rush_Hour']
target = 'Rented_Bike_Count'

# 범주형 특성 목록
categorical_features_final = ['Seasons', 'Holiday', 'Functioning_Day']

X_final = df_final[features]
y_final = df_final[target]

# 학습/테스트 데이터 분리
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

### Step 2: 3개 모델 학습 및 평가

아래 가이드에 따라 XGBoost, LightGBM, CatBoost 모델을 각각 학습시키고, 결과를 `results` 리스트에 저장하세요.

  * **XGBoost**: `pd.get_dummies`로 범주형 변수를 원-핫 인코딩한 데이터를 사용해야 합니다.
  * **LightGBM**: `pd.get_dummies`로 인코딩된 데이터를 사용하거나, `pandas.Categorical` 타입으로 변환하여 직접 처리할 수 있습니다. 여기서는 XGBoost와 동일하게 인코딩된 데이터를 사용하겠습니다.
  * **CatBoost**: 원-핫 인코딩되지 않은 원본 형태의 데이터를 사용하고, `cat_features` 인자를 꼭 설정해주세요.

<!-- end list -->

In [ ]:
# 결과 저장을 위한 리스트
results = []

# --- 1. XGBoost ---
# XGBoost를 위한 데이터 전처리 (원-핫 인코딩)
X_train_xgb_f = pd.get_dummies(X_train_f, columns=categorical_features_final, drop_first=True)
X_test_xgb_f = pd.get_dummies(X_test_f, columns=categorical_features_final, drop_first=True)

# 모델 학습
xgb_final = xgb.XGBRegressor(random_state=42, n_jobs=-1)
xgb_final.fit(X_train_xgb_f, y_train_f)
y_pred_xgb_f = xgb_final.predict(X_test_xgb_f)

# 성능 계산 및 저장
rmse_xgb_f = np.sqrt(mean_squared_error(y_test_f, y_pred_xgb_f))
r2_xgb_f = r2_score(y_test_f, y_pred_xgb_f)
results.append({'Model': 'XGBoost', 'RMSE': rmse_xgb_f, 'R2': r2_xgb_f})


# --- 2. LightGBM ---
# LightGBM은 XGBoost와 동일한 데이터 사용
lgbm_final = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
lgbm_final.fit(X_train_xgb_f, y_train_f) # XGBoost용으로 인코딩된 데이터 사용
y_pred_lgbm_f = lgbm_final.predict(X_test_xgb_f)

# 성능 계산 및 저장
rmse_lgbm_f = np.sqrt(mean_squared_error(y_test_f, y_pred_lgbm_f))
r2_lgbm_f = r2_score(y_test_f, y_pred_lgbm_f)
results.append({'Model': 'LightGBM', 'RMSE': rmse_lgbm_f, 'R2': r2_lgbm_f})


# --- 3. CatBoost ---
# CatBoost는 원본 데이터 사용
cat_final = cb.CatBoostRegressor(random_state=42, cat_features=categorical_features_final, verbose=0)
cat_final.fit(X_train_f, y_train_f) # 인코딩 전 데이터 사용
y_pred_cat_f = cat_final.predict(X_test_f)

# 성능 계산 및 저장
rmse_cat_f = np.sqrt(mean_squared_error(y_test_f, y_pred_cat_f))
r2_cat_f = r2_score(y_test_f, y_pred_cat_f)
results.append({'Model': 'CatBoost', 'RMSE': rmse_cat_f, 'R2': r2_cat_f})

# 결과 데이터프레임 생성
results_df = pd.DataFrame(results)
print(results_df)

### Step 3: 결과 비교 및 시각화

`plotly.express`를 사용하여 세 모델의 RMSE와 R² 점수를 막대그래프로 비교해 봅시다.

In [ ]:
# RMSE 비교 시각화
fig_rmse = px.bar(results_df.sort_values('RMSE'),
                  x='Model',
                  y='RMSE',
                  title='Model Comparison: RMSE (Lower is Better)',
                  color='Model',
                  text_auto='.4f')
fig_rmse.show()

# R² Score 비교 시각화
fig_r2 = px.bar(results_df.sort_values('R2', ascending=False),
                x='Model',
                y='R2',
                title='Model Comparison: R² Score (Higher is Better)',
                color='Model',
                text_auto='.4f')
fig_r2.show()

### Step 4: 결과 해석 (자율 양식)

  * 어떤 모델이 가장 좋은 성능(가장 낮은 RMSE, 가장 높은 R²)을 보였나요?
  * 각 모델의 학습 시간은 어땠나요? (코드를 수정하여 측정해보세요.)
  * 이 데이터셋의 특성(범주형 변수의 존재, 데이터 크기 등)을 고려할 때, 왜 특정 모델이 더 좋은 성능을 보였을지 자신의 생각을 정리해보세요.
  * 기타 생각해볼 점이 있다면 적어보세요.